In [26]:
from dotenv import load_dotenv


load_dotenv()

True

In [1]:
import requests

# Nominatim Base URL
BASE_URL = "https://nominatim.openstreetmap.org/"

# Function for Forward Geocoding (Address to Coordinates)
def forward_geocode(address, format="json"):
    params = {
        'q': address,
        'format': format,
        'addressdetails': 1,
        'limit': 1
    }
    response = requests.get(BASE_URL + "search", params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Failed to fetch data"}

# Function for Reverse Geocoding (Coordinates to Address)
def reverse_geocode(lat, lon, format="json"):
    params = {
        'lat': lat,
        'lon': lon,
        'format': format,
        'addressdetails': 1
    }
    response = requests.get(BASE_URL + "reverse", params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Failed to fetch data"}

# Example Usage
if __name__ == "__main__":
    address = "1600 Amphitheatre Parkway, Mountain View, CA"
    lat, lon = 37.4221, -122.0841

    # Forward Geocode Example
    print("Forward Geocode:")
    result = forward_geocode(address)
    print(result)

    # Reverse Geocode Example
    print("\nReverse Geocode:")
    result = reverse_geocode(lat, lon)
    print(result)


Forward Geocode:
[{'place_id': 376947505, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 2192620021, 'lat': '37.4217636', 'lon': '-122.084614', 'class': 'office', 'type': 'it', 'place_rank': 30, 'importance': 0.6949356759210291, 'addresstype': 'office', 'name': 'Google Headquarters', 'display_name': 'Google Headquarters, 1600, Amphitheatre Parkway, Mountain View, Santa Clara County, California, 94043, United States', 'address': {'office': 'Google Headquarters', 'house_number': '1600', 'road': 'Amphitheatre Parkway', 'city': 'Mountain View', 'county': 'Santa Clara County', 'state': 'California', 'ISO3166-2-lvl4': 'US-CA', 'postcode': '94043', 'country': 'United States', 'country_code': 'us'}, 'boundingbox': ['37.4217136', '37.4218136', '-122.0846640', '-122.0845640']}]

Reverse Geocode:
{'place_id': 281801698, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id':

In [6]:
from ratelimit import limits, sleep_and_retry

import requests

@sleep_and_retry
@limits(calls=1, period=1)
def call_api():
    print("test call!")

In [7]:
for i in range(10):
    call_api()

test call!
test call!
test call!
test call!
test call!
test call!
test call!
test call!
test call!
test call!
